In [1]:
!pip install langchain_openai langchain_community pypdf chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 719.1 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:

In [3]:
def get_key():
  with open('20250723_OpenAI_API.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr

os.environ['OPENAI_API_KEY'] = get_key()


In [5]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")

loader = PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()
print('문서의 수 :', len(texts))


문서의 수 : 366


In [6]:
texts[15]


Document(metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 19, 'page_label': '20'}, page_content='3\nㄱ \n신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통\n계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 \n부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .\n가계처분가능소득 \n가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축\n으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 \n지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 \n정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 \n보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 \n체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal \nGross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누

In [7]:
print(texts[15].page_content)


3
ㄱ 
신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통
계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 
부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .
가계처분가능소득 
가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축
으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 
지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 
정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 
보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 
체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal 
Gross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누어 계산한 
지표로 가계의 구매력을 가장 정확히 가늠해 볼 수 있는 소득지표이다 . 
 연관검색어 : 국민총소득 (GNI)
가교은행
청산 대상 금융기관의 자산, 부채를 임시로 넘겨받아 예금, 출금 등의 업무를 대행하고 
합병, 채권채무관계 조정 등 후속조치를 수행하는 은행을 말한다. 금융기관이 파산한 경우 
예금보험제도는 청산, 매각, 자산부채승계, 가교은행을 통한 인수 등의 수단을 통해 부실 
금융기관을 처리한다. 이 중 가교은행을 통한 인수방식은 파산은행의 처리에 상당한 시간이 
소요될 것으로 예상할 경우 사용되며 새로운 은행을 설립하여 자산, 부채를 포괄승계하도록 
하고, 인수 희망자를 물색하는 등의 조치를 취하게 된다. 가교은행을 이용한 처리방식은 
금융기관 파산에 따른 충격을 완화하고 시간적인 여유를 확보할 수 있는 것이 장점이다. 
임무를 마치면 정리대상 금융기관과 마찬가지로 없어지는 한시적 기관이다.
가동률
생산능력 대비 생산실적의 백분율(생산실적/생산능력×100)로

In [8]:
print(texts[0].page_content)


iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 
시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 
사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 
독자들의 편의성과 가독성을 높였습니다 . 
코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 
일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 
정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 
경제금융용어 700선
  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 
의견을 주신 관련부서 여러분께 감사드립니다 .
2020년 8월
한국은행 경제교육실장 박 철 원
머 리 말


In [9]:
print(texts[5].page_content)


vi
경제금융용어 700선
ㅂ
바이오인증 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
바젤은행감독위원회/
바젤위원회(BCBS) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
발행시장 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・119
발행중지화폐/유통정지화폐 ・・・・・・・・・・・・・・・120
방카슈랑스 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・120
배당할인모형 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
밴드웨건효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
뱅 크 런 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・122
범위의 경제 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
법률리스크 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
베블런효과 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・123
변동금리 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・123
변동금리부채권(FRN) ・ ・ ・・・・・・・・・・・・・・・・・・・・・・・123
보기화폐(견양화폐) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・124
보완자본(Tier 2) ・・・・・・・・・・・

In [10]:
print(texts[13].page_content)


1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [11]:
texts = texts[13:]
print('줄어든 청크의 개수:', len(texts))


줄어든 청크의 개수: 353


In [12]:
print('첫번째 청크 출력 :', texts[0])


첫번째 청크 출력 : page_content='1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙' metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'ti

In [13]:
print('마지막 청크 출력 :', texts[-2])


마지막 청크 출력 : page_content='352
경제금융용어 700선
와 관련된 다양한 부수업무를 하는 회사를 말한다. 가령 신용카드 거래가 발생하면 
VAN사업자는 거래 승인을 위해 신용카드사에 거래내역을 전송하고 승인 및 조회결과를 
수신한 후 이를 가맹점에 전송한다. 또한 VAN사업자는 가맹점이 신용카드사에 전표를 
제출하여 대금을 청구하는 업무를 대신하기 위하여 가맹점을 방문하여 전표실물을 수거
하거나 전표 데이터를 수집한다 . 
 연관검색어 : 지급결제시스템, 전자금융
VaR(Value at Risk) 
주어진 신뢰수준 하에서 일정 기간 동안 발생할 수 있는 ‘최대 손실금액’으로 금융기관
의 잠재적인 손실을 측정하는 지표이다. 예를 들어 목표기간 1년, 신뢰수준 95% 에서 
산출된 VaR가 10억 원이라면 이는 1년 동안 발생할 수 있는 손실금액이 10억 원보다 
작을 확률이 95%라는 것을 의미한다 . 
 연관검색어 : 예상손실
VIX
미국 주식시장의 단기 변동성에 대한 시장의 기대치를 나타내는 지수로 시카고옵션거
래소(CBOE)에서 제공되며, 정식명칭은 CBOE Volatility Index이다. VIX는 향후 30 일 
동안의 S&P 500 지수의 변동성에 대한 시장의 기대치로서, 지수의 변동성이 클 것으로 
예상될 경우 옵션가격이 높아지는 점에 착안하여 CBOE에 상장된 다양한 행사가격의 
S&P 500 지수 옵션들의 가격을 활용하여 산출된다. VIX는 일반적으로 기초자산 가격과 
음(-)의 상관관계가 있다. 예를 들어 주가지수가 상승할 때 하락하고 주가지수가 하락할 
때는 상승한다. 이에 따라 VIX의 상승은 투자자들의 불안심리가 증대하는 것을 의미하므
로 공포지수라고도 부른다. 1993년 Robert E. Whaley 교수의 논문에서 처음 소개된 
이후 시장의 변동성과 투자자들의 심리를 나타내주는 주요한 지표로 널리 활용되고 
있다. 이 외의 대표적인 주식시장 변동성 지수로는 유럽의 VSTOXX 등이 있다. VSTOXX
는 유럽의 대

In [14]:
texts = texts[:-1]
print('마지막 데이터 제거 후 청크의 개수:', len(texts))

마지막 데이터 제거 후 청크의 개수: 352


In [15]:
embedding = OpenAIEmbeddings(chunk_size=100)

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)


In [16]:
print(vectordb._collection.count())


352


In [17]:
for key in vectordb._collection.get():
  print(key)


ids
embeddings
documents
uris
included
data
metadatas


In [18]:
documents = vectordb._collection.get()['documents']
print('청크의 개수 :', len(documents))
print('-' * 50)
print('0번 청크 출력 :', documents[0])


청크의 개수 : 352
--------------------------------------------------
0번 청크 출력 : 1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [19]:
embeddings = vectordb._collection.get(include=['embeddings'])['embeddings']
print('임베딩 벡터의 개수 :', len(embeddings))


임베딩 벡터의 개수 : 352


In [20]:
print('0번 청크의 임베딩 값 출력 :', embeddings[0])
print('0번 청크의 임베딩 값의 길이 :', len(embeddings[0]))


0번 청크의 임베딩 값 출력 : [-0.0077088  -0.0075166   0.014182   ... -0.01861645 -0.00669286
 -0.00903364]
0번 청크의 임베딩 값의 길이 : 1536


In [21]:
metadatas = vectordb._collection.get()['metadatas']
print('metadatas의 개수 :', len(metadatas))
print('0번 청크의 출처 :', metadatas[0])


metadatas의 개수 : 352
0번 청크의 출처 : {'moddate': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'creator': 'Smart PDF Imposition 1.0', 'producer': 'Smart PDF Imposition 1.0', 'moddate--text': 'January 31, 2018 16:21', 'total_pages': 371, 'source': '2020_경제금융용어 700선_게시.pdf', 'creationdate--text': 'January 31, 2018 16:21', 'page': 17, 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'page_label': '18'}


In [22]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

docs = retriever.get_relevant_documents("비트코인이 궁금해")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('두번째 유사 문서 :', docs[1])


/tmp/ipython-input-22-1996083593.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("비트코인이 궁금해")


유사 문서 개수 : 2
----------------------------------------
첫번째 유사 문서 : page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다

In [23]:
template = """당신은 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이입니다.
안상준 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)


In [24]:
llm = ChatOpenAI(model_name="gpt-4.1", temperature=0)


In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever,
    return_source_documents=True)


In [26]:
input_text = "디커플링이란 무엇인가?"
chatbot_response = qa_chain(input_text)
print(chatbot_response)


/tmp/ipython-input-26-1641367228.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatbot_response = qa_chain(input_text)


{'query': '디커플링이란 무엇인가?', 'result': '디커플링(decoupling)은 탈동조화라고 하는데, 어떤 나라나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다르게 움직이는 현상을 말해. 예를 들어, 미국 경제가 침체인데 신흥국 경제는 잘 나간다거나, 미국 주가는 오르는데 한국 주가는 내리는 식으로 서로 다른 방향으로 움직이는 거지. 또 한 나라 안에서도 주가는 떨어지는데 그 나라 통화는 강세를 보이는 경우도 디커플링이라고 할 수 있어. 반대로 경제 흐름이 비슷하게 움직이면 그건 커플링(동조화)이라고 해.', 'source_documents': [Document(metadata={'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'author': 'INSUNG DATA INC.', 'source': '2020_경제금융용어 700선_게시.pdf', 'creationdate': 'January 31, 2018 16:21', 'creator': 'Smart PDF Imposition 1.0', 'page_label': '115', 'producer': 'Smart PDF Imposition 1.0', 'total_pages': 371, 'creationdate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'page': 114}, page_content='98\n경제금융용어 700선\n가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 \n보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 예상치가 더 \n낮아지지 않도록 사전에 통화 ･재정정책 면에서 적극적으로 대응하는 것이 중요하다 .\n 연관검색어 : 디플레이션,  통화정책

In [27]:
def get_chatbot_response(input_text):
    chatbot_response = qa_chain(input_text)
    return chatbot_response['result'].strip()


In [28]:
input_text = "너는 뭘하는 챗봇이니?"
result = get_chatbot_response(input_text)
print(result)


나는 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이야. 궁금한 금융 용어가 있으면 검색 결과를 바탕으로 친근하게 설명해줄게. 검색 결과에 없는 내용은 답변할 수 없어! 궁금한 거 있으면 언제든 물어봐~


In [29]:
input_text = "비트코인에 대해서 궁금하당~"
result = get_chatbot_response(input_text)
print(result)


비트코인에 대해 궁금하구나! 비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이야. 중앙 저장소나 단일 관리자가 없어서, 최초의 탈중앙화된 디지털통화라고 불려. 2009년에 사토시 나카모토라는 사람이(혹은 집단이) 만들었고, 오픈소스로 배포됐지.

비트코인 시스템은 공유형(P2P) 네트워크로 운영돼. 거래는 중개자 없이 참여자들끼리 직접 이뤄지고, 이 거래들은 작업증명(proof-of-work)이라는 과정을 거쳐 블록체인이라는 분산원장에 기록돼. 새로운 거래 그룹(블록)은 약 10분마다 만들어져서 블록체인에 추가되고, 모든 참여자에게 바로 전파돼.

비트코인은 채굴(mining)이라는 과정을 통해 발행돼. 이 채굴은 대규모 전기를 쓰는 컴퓨터 처리 능력을 활용해서 이뤄지고, 채굴자들은 그 보상으로 비트코인을 받아. 비트코인은 법정화폐가 아니라서, 비트코인 플랫폼에서만 거래되고, 투자 대상으로도 인기가 많아. 하지만 가격 변동이 심해서 거품 논란도 크고, 익명성 때문에 자금세탁 등 불법거래에 악용될 소지도 있어서 일부 국가는 거래를 규제하고 있어.

비트코인의 총량은 2,100만 개로 한정돼 있고, 2140년쯤이면 모두 채굴될 거라고 해. 그래서 인플레이션에 의한 가치 하락 위험이 적은 편이야. 참고로, 비트코인에서 파생된 비트코인캐시(BCH), 비트코인골드(BTG) 같은 것도 있어.

궁금한 거 더 있으면 언제든 물어봐!


In [88]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 5.7 MB/s eta 0:00:00
  Attempting uninstall: gradio
    Found existing installation: gradio 5.38.0
    Uninstalling gradio-5.38.0:
      Successfully uninstalled gradio-5.38.0


In [35]:
!python --version

Python 3.11.13


In [38]:
def greet(name):
  return "Hello " + name + "!"

greet("World")

'Hello World!'

In [39]:
gr.Interface(greet, "text", "text").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5576c164841c0a6b94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [37]:
import time

# 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages", label="경제금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창 레이블
    clear = gr.ClearButton([msg, chatbot], value="대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가
      chat_history.append({"role": "user", "content": message})
      chat_history.append({"role": "assistant", "content": bot_message})
      time.sleep(2)
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

# 인터페이스 실행
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3fc17a49b4c2bca99a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


20151201_GS1 Information 5_유통물류 국제표준 용어집

In [51]:
loader = PyPDFLoader("20151201_GS1 Information 5_유통물류 국제표준 용어집.pdf")
texts_new = loader.load_and_split()
print('문서의 수 :', len(texts_new))

문서의 수 : 106


In [52]:
texts_new[15]

Document(metadata={'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'creationdate': '2015-12-15T17:12:31+09:00', 'moddate': '2015-12-15T17:22:55+09:00', 'trapped': '/False', 'source': '20151201_GS1 Information 5_유통물류 국제표준 용어집.pdf', 'total_pages': 107, 'page': 15, 'page_label': '16'}, page_content='15 | C\n<카테고리관리 비즈니스 프로세스 활동>\nCheck Digit \n식별코드의 마지막에 위치하는 1자리의 숫자. 이전 숫자들이 올바르게 부여되었는지 검증하는 역할을 \n한다. 국가코드, 업체코드, 상품코드를 이용하여 특정 공식에 따라 계산하여 결정된다(아래 계산법 참조). \n임의로 부여할 경우 해당 바코드(식별코드)는 판독이 불가능하다. 대한상공회의소 유통물류진흥원에서 \n운영하는 코리안넷(www.koreannet.or.kr) 홈페이지에 있는 체크디지트 계산 프로그램을 통해 체크디지\n트를 자동으로 계산할 수 있다. \n<체크디지트 계산법>\n자리수\nGTIN-8 N1 N2 N3 N4 N5 N6 N7 N8 N9\nGTIN-13 N1 N2 N3 N4 N5 N6 N7 N8 N9 N10 N11 N12 N13\nGTIN-14 N1 N2 N3 N4 N5 N6 N7 N8 N9 N10 N11 N12 N13 N14\nSSCC N1 N2 N3 N4 N5 N6 N7 N8 N9 N10 N11 N12 N13 N14 N15 N16 N17 N18\n  각 자리수에 다음과 같이 3과 1을 번갈아서 곱함\nx3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 C/D\n  각 자리수에 3과 1을 곱하여 나온 갑을 모두 더함 = 총합\

In [53]:
print(texts_new[15].page_content)


15 | C
<카테고리관리 비즈니스 프로세스 활동>
Check Digit 
식별코드의 마지막에 위치하는 1자리의 숫자. 이전 숫자들이 올바르게 부여되었는지 검증하는 역할을 
한다. 국가코드, 업체코드, 상품코드를 이용하여 특정 공식에 따라 계산하여 결정된다(아래 계산법 참조). 
임의로 부여할 경우 해당 바코드(식별코드)는 판독이 불가능하다. 대한상공회의소 유통물류진흥원에서 
운영하는 코리안넷(www.koreannet.or.kr) 홈페이지에 있는 체크디지트 계산 프로그램을 통해 체크디지
트를 자동으로 계산할 수 있다. 
<체크디지트 계산법>
자리수
GTIN-8 N1 N2 N3 N4 N5 N6 N7 N8 N9
GTIN-13 N1 N2 N3 N4 N5 N6 N7 N8 N9 N10 N11 N12 N13
GTIN-14 N1 N2 N3 N4 N5 N6 N7 N8 N9 N10 N11 N12 N13 N14
SSCC N1 N2 N3 N4 N5 N6 N7 N8 N9 N10 N11 N12 N13 N14 N15 N16 N17 N18
  각 자리수에 다음과 같이 3과 1을 번갈아서 곱함
x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 x1 x3 C/D
  각 자리수에 3과 1을 곱하여 나온 갑을 모두 더함 = 총합
체크디지트   10의 배수 중 총합보다 크면서 총합에 가장 가까운 수에서 총합을 뺌


In [54]:
print(texts_new[4].page_content)

A | 4
Glossary of Global Standard Terms on Distribution and Logistics
ABC (Activity Based Costing) 
활동기준원가회계. 간접비를 직접비로 전환하는데 사용되는 회계기법이다. 전통적인 원가계산방식은 간
접지원비용(예: 구매주문, 원재료 처리, 생산계획, 작업준비, 품질검사 등)을 인위적인 기준에 의해서 배
분함으로써 제품이나 서비스 원가를 왜곡하였다. ABC는 이러한 허점을 보완하고 보다 정확한 원가계산
을 위해 고안된 기법으로서, 기업의 기능을 여러 가지 활동들(Activities)1)로 구분한 다음 각 활동을 기본
적인 원가대상으로 삼아 원가를 집계하고 이를 토대로 하여 다른 대상들의 원가를 산정한다. ECR/SCM
에서 ABC가 중요한 위치를 차지하는 이유는 ECR/SCM 활동을 통해 창출된 이익을 참여 당사자 간에 어
떻게 분배할 것인가를 결정하는데 매우 유용하게 사용되기 때문이다.
Active Tag
능동형 태그. 외부 무선 신호 없이 자체 무선 신호를 발생시킬 수 있는 능력을 갖고 있는 전자태그(RFID) 
장치를 지칭한다. 능동형 태그가 무선 신호를 발생시키기 위해서는 전력원이 있어야 하는데, 태양 에
너지나 주변 환경의 대기 에너지를 이용하는 장치도 있긴 하지만 전통적으로 집적 배터리(Integrated 
Batteries)가 사용되고 있다. 
[비교] Passive Tag
ADC (Automatic Data Capture) 
자동식별 및 데이터인식 기술. 사람이 키보드 등 입력장치를 사용하지 않고 데이터를 컴퓨터 시스템에 
입력하는 것을 지칭한다. ADC 기술은 개체 식별뿐만 아니라 개체 추적에도 사용되는 신뢰성이 검증된 
수단이며 개체의 식별 정보 외에도 대량의 상세 정보까지 입력할 수 있다. ADC에는 바코드, RFID, 자기 
선, 카드 기술, 광학문자판독(OCR) 등이 포함된다.
A
1)  기업 내의 각 집단이 기업 목표 달성을 위하여 수행하는 일체의 반복적인 과업

In [55]:
texts_new = texts_new[4:]
print('줄어든 청크의 개수:', len(texts_new))

줄어든 청크의 개수: 102


In [56]:
print('첫번째 청크 출력 :', texts_new[0])


첫번째 청크 출력 : page_content='A | 4
Glossary of Global Standard Terms on Distribution and Logistics
ABC (Activity Based Costing) 
활동기준원가회계. 간접비를 직접비로 전환하는데 사용되는 회계기법이다. 전통적인 원가계산방식은 간
접지원비용(예: 구매주문, 원재료 처리, 생산계획, 작업준비, 품질검사 등)을 인위적인 기준에 의해서 배
분함으로써 제품이나 서비스 원가를 왜곡하였다. ABC는 이러한 허점을 보완하고 보다 정확한 원가계산
을 위해 고안된 기법으로서, 기업의 기능을 여러 가지 활동들(Activities)1)로 구분한 다음 각 활동을 기본
적인 원가대상으로 삼아 원가를 집계하고 이를 토대로 하여 다른 대상들의 원가를 산정한다. ECR/SCM
에서 ABC가 중요한 위치를 차지하는 이유는 ECR/SCM 활동을 통해 창출된 이익을 참여 당사자 간에 어
떻게 분배할 것인가를 결정하는데 매우 유용하게 사용되기 때문이다.
Active Tag
능동형 태그. 외부 무선 신호 없이 자체 무선 신호를 발생시킬 수 있는 능력을 갖고 있는 전자태그(RFID) 
장치를 지칭한다. 능동형 태그가 무선 신호를 발생시키기 위해서는 전력원이 있어야 하는데, 태양 에
너지나 주변 환경의 대기 에너지를 이용하는 장치도 있긴 하지만 전통적으로 집적 배터리(Integrated 
Batteries)가 사용되고 있다. 
[비교] Passive Tag
ADC (Automatic Data Capture) 
자동식별 및 데이터인식 기술. 사람이 키보드 등 입력장치를 사용하지 않고 데이터를 컴퓨터 시스템에 
입력하는 것을 지칭한다. ADC 기술은 개체 식별뿐만 아니라 개체 추적에도 사용되는 신뢰성이 검증된 
수단이며 개체의 식별 정보 외에도 대량의 상세 정보까지 입력할 수 있다. ADC에는 바코드, RFID, 자기 
선, 카드 기술, 광학문자판독(OCR) 등이 포함된다.
A
1)  기업 내의 각 집단이 기업 목

In [57]:
print('마지막 청크 출력 :', texts_new[-21])


마지막 청크 출력 : page_content='85 | 색인
A
ABC (Activity Based Costing)  04
Active Tag 04
ADC (Automatic Data Capture)  04
Add-On Symbol 05
Adjacent Symbol 05
After Service 05
Agile Reader 05
AI (Application Identifier) 05
AIDC (Automatic Identification and Data Capture) 07
AIM (Association for Automatic Identification and Mobility) 07
Alphanumeric 07
ANSI (American National Standards Association) 07
Aperture 08
APS (Advanced Planning and Scheduling) 08
ASC (Accredited Standards Committee) 08
ASCII (America Standard Code for Information Integration) 08
ASN (Advanced Shipping Notice)   08
Asset Type 09
AS2 (Applicability Statement 2) 09
Attribute 09
Autodiscrimination 09
Auto-ID Center 09
Available Inventory 10
A2A (Application-to-Application) 10
B
Backroom 11
Bar Gain/Loss 11
Barcode 11
색인' metadata={'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'creationdate': '2015-12-15T17:12:31+09:00', 'moddate': '2015-12-15T17:22:55+09:00', 'trapped': '/False', 'source': '

In [58]:
texts_new = texts_new[:-21]
print('마지막 데이터 제거 후 청크의 개수:', len(texts_new))

마지막 데이터 제거 후 청크의 개수: 81


In [59]:
print('마지막 청크 출력 :', texts_new[-1])

마지막 청크 출력 : page_content='2-3 | 84
Glossary of Global Standard Terms on Distribution and Logistics
2-Dimensional Barcode 
2차원 바코드. 어두운 바(Bar)와 밝은 바(Space)를 조그마한 사각형 안에 가로와 세로 (x, y) 양방향으로 
점자, 모자이크식 또는 다층으로 표현한 평면 바코드. 따라서 기존 1차원 바코드보다 훨씬 많은 고밀도 
정보를 담을 수 있다. 어느 방향으로 스캐너를 대도 인식되고, 정보가 훼손되어도 상당 부분 복구가 가능
하며, 기존 바코드와 달리 데이터베이스 없이도 그 자체로 해당 정보를 파악할 수 있다. 바코드 자체에 문
자, 숫자 등 텍스트는 물론, 그래픽, 사진, 음성, 지문, 서명 등 다양한 형태의 정보를 저장할 수 있다.
32
3PL (3rd Party Logistics)
3자 물류. 다양한 물류 채널 내 다른 주체와의 일시적이거나 장기적인 관계를 가지고 있는 물류 채널내
의 대행자 또는 매개자를 의미한다. 다시 말해 화주와 단일 혹은 복수의 3PL 간에 일정 기간 동안 일정 
비용으로 일정 서비스를 상호 합의 하에 수행하는 과정을 3PL이라고 정의하고 있다. 1988년 미국물류관
리협회에서 처음 사용한 용어로 기업이 사내에서 수행하던 물류기능을 아웃소싱하는 경향이 증가하면서 
물류분야의 핵심적인 개념으로 자리 잡았다.' metadata={'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'creationdate': '2015-12-15T17:12:31+09:00', 'moddate': '2015-12-15T17:22:55+09:00', 'trapped': '/False', 'source': '20151201_GS1 Information 5_유통물류 국제표준 용어집.pdf', 'total_pages': 107, '